## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_imputation(df):
    data = df.copy()
    data = data.drop(columns=["ID","CLASS"])
    flist = list(data.select_dtypes(include=['float','int']).columns)
    clist = list(data.select_dtypes(include=['category']).columns)
    olist = list(data.select_dtypes(include=['object']).columns)
    
    dicti={}
    for c in flist:
        if (data[c].isnull().all() == True):
            data[c].fillna(0,inplace=True)
            dicti[c]=0
        else:
            data[c].fillna(data[c].mean(),inplace=True)
            dicti[c]=data[c].mean()
    for k in clist:
        if (data[k].isnull().all() == True):
            data[k].fillna(data[k].cat.categories[0],inplace=True)
            dicti[k]=data[k].cat.categories[0]
        else:
            data[k].fillna(data[k].mode()[0],inplace=True)
            dicti[k] = data[k].mode()[0]
    for z in olist:
        if (data[z].isnull().all() == True):
            data[z].fillna("",inplace=True)
            dicti[z] = ""
        else:
            data[z].fillna(data[z].mode()[0],inplace=True)
            dicti[z] = data[z].mode()[0]
    data = data.assign(CLASS=df["CLASS"].values)
    data = data.assign(ID=df["ID"].values)
    return data,dicti

def apply_imputation(df,dicti):
    test = df.copy()
    test = test.drop(columns=["ID","CLASS"])
    flist_t = list(test.select_dtypes(include=['float','int']).columns)
    clist_t = list(test.select_dtypes(include=['category']).columns)
    olist_t = list(test.select_dtypes(include=['object']).columns)
    for c in flist_t:
        if (test[c].isnull().all() == True):
            test[c].fillna(0,inplace=True)
        else:
            test[c].fillna(dicti[c],inplace=True)
    for k in clist_t:
        if (test[k].isnull().all() == True):
            test[k].fillna(test[k].cat.categories[0],inplace=True)
        else:
            test[k].fillna(dicti[k],inplace=True)
    for z in olist_t:
        if (test[z].isnull().all() == True):
            test[z].fillna("",inplace=True)
        else:
            test[z].fillna(dicti[z],inplace=True)
    test = test.assign(CLASS=df["CLASS"].values)
    test = test.assign(ID=df["ID"].values)
    return test

def create_normalization(data,normalizationtype="minmax"):
    df = data.copy() # create a copy of the data
    df = df.drop(columns=["ID","CLASS"])
    df.select_dtypes(include=['float','int'])
    if(normalizationtype == "minmax"):
        dicti = {}
        for x in df.columns:
            mini = df[x].min()
            maxi = df[x].max()
            df[x] = [(y - mini)/(maxi-mini) for y in df[x]]
            dicti[x] =("minmax",mini,maxi)
        df = df.assign(CLASS=data["CLASS"].values)
        return df,dicti
    elif(normalizationtype =="zscore"):  
        dict2 = {}
        for x in df.columns:
            meane = df[x].mean()
            stdi = df[x].std()
            df[x] = df[x].apply(lambda x:(x-meane)/stdi)
            dict2[x] =("zscore",meane,stdi)
        df = df.assign(CLASS=data["CLASS"].values)
        return df,dict2
    
def apply_normalization(test,normalization):
    dft = test.copy()
    dft = dft.drop(columns=["ID"])
    flist = list(dft.select_dtypes(include=['float']).columns)
    for c in flist:
        m = normalization[c][1] #m represents the minimum value from the normalization dictionary
        h = normalization[c][2] #h represents the maximum value from the normalization dictionary
        dft[c] = [(y - m)/(h-m) for y in dft[c]]
    return dft

def euclidean(arrone,arrtwo):
    summ= 0
    for x in range(len(arrone)):
        diff = (arrone[x] - arrtwo[x])
        squared = diff * diff
        summ = squared + summ
    return np.sqrt(summ)
def SortValue(value):
    return value[1]
def getclass(sorti,trainlabels):
    #The function returns the class labels for the K number of classes.
    classes = []
    for x in range(len(sorti)):
        classes.append(trainlabels[sorti[x][0]]) 
    return classes
def highestclass(K_classes):
    #The function returns a 2-D array with the class and its occurences/count number
    unique, counts = np.unique(K_classes, return_counts=True)
    final = []
    for i in range(len(unique)):
        final.append([unique[i],counts[i]])
    return final

def finalclass(arrone):
    #The function returns the class with the highest occurence as the reverse value is true(descending)
    finalarr = sorted(arrone,key=SortValue,reverse=True)
    fin = finalarr[0][0]
    return fin

def thresholdfinder(prediction):
    val = list(prediction.unique())
    val.sort(reverse=True)
    x = val[0] + 1
    val.insert(0,x)
    return val

def CMatrix(correctlabels, predictions):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(predictions)): 
        if (correctlabels[i]==predictions[i]==1):
            TP += 1
        if (predictions[i]==1 and correctlabels[i]!=predictions[i]):
            FP += 1
        if (correctlabels[i]==predictions[i]==0):
            TN += 1
        if (predictions[i]==0 and correctlabels[i]!=predictions[i]):
            FN += 1
    return TP, FP, TN, FN

def labeltransformertwo(label,labels):
    binary = []
    for k in range(len(labels)):
        if (labels[k] == label):
            binary.append(1)
        else:
            binary.append(0)        
    return binary
def threshpredictor(threshold,predictions):
    finalpred = [1 if x >= threshold else 0 for x in predictions]
    return finalpred
def tprfinder(thresholds,predictions,ytransformed):
    tpr_arr =[]
    fpr_arr =[]
    for x in thresholds:
        arr = threshpredictor(x,predictions)
        TP, FP, TN, FN = CMatrix(ytransformed,arr)
        TPR = TP/(TP + FN)
        FPR = FP/(TN + FP)
        tpr_arr.append(TPR)
        fpr_arr.append(FPR)
    return tpr_arr,fpr_arr
    
def class_couter(correctlabels,label):
    count = 0
    for item in correctlabels:
        if(item == label):
            count += 1
    return count

def calc_indv_auc(tpr, fpr):
    zero = np.array([0])
    tpr_diff = np.hstack((np.diff(tpr), zero))
    fpr_diff = np.hstack((np.diff(fpr), zero))
    auc = np.dot(tpr, fpr_diff) + np.dot(tpr_diff, fpr_diff) / 2
    return auc
def auc(predictions,correctlabels):
    AUC_ ={}
    for item in predictions.columns:
        y = labeltransformertwo(item,correctlabels)
        thresh = thresholdfinder(predictions[item]) 
        tpr_arr,fpr_arr = tprfinder(thresh,predictions[item],y)
        AUC_[item] = calc_indv_auc(tpr_arr,fpr_arr) 
        
    aucs = []
    for name in AUC_:
        frequecy = class_couter(correctlabels,name)
        weighted_auc = AUC_[name]*frequecy
        aucs.append(weighted_auc)
    auc = (np.sum(aucs)/len(predictions))
    return auc

def accuracy(predictions,correctlabels):
    count = 0
    numrows = len(correctlabels)
    for x,y in zip(predictions.index, range(numrows)):
        pos = np.argmax(np.array(predictions.loc[x]))
        if(predictions.columns[pos] == correctlabels[y]):
            count += 1
    accuracy = count/numrows
    return accuracy

def brier_score(df,correctlabels):
    col_list = list(df.columns)
    rw = 0
    b_score = 0
    for index, row in df.iterrows():
        for col in col_list:
            if col == correctlabels[rw]:
                val = 1
                b = (row[col]-val)**2
            else:
                val = 0
                b = (row[col]-val)**2
            b_score = b_score + b
        rw+=1
    b_score = b_score/(len(df))
    return b_score  

In [3]:
class kNN():
    def __init__(self):
        pass
        
    def fit(self,df,normalizationtype="minmax"):
        self.imputed,self.imputer = create_imputation(df)
        self.normalized,self.normalizer = create_normalization(self.imputed,normalizationtype)
        self.traininglabels = self.normalized["CLASS"].astype('category')
        self.labels = list(self.normalized["CLASS"].unique())
        self.labels.sort()
        self.labels = np.array(self.labels)
        self.trainingdata = self.normalized.drop(columns=["CLASS"],axis=1)
        self.trainingdata = np.array(self.trainingdata)
    def predict(self,df,k=1):
        self.testimp = apply_imputation(df,self.imputer)
        self.testnorm = apply_normalization(self.testimp,self.normalizer)
        self.testnorm = self.testnorm.drop(columns=["CLASS"],axis=1)
        self.testnorm = np.array(self.testnorm)
        
        creator = np.zeros(shape=(len(self.testnorm),len(self.labels)))
        data = pd.DataFrame(creator,columns=self.labels)
        incre = 0
        predicted = []
        for row in self.testnorm:
            indexi = 0
            sorti =[]
            dist_values = []
            for rowe in self.trainingdata:
                dist = euclidean(row,rowe)
                dist_values.append([indexi,dist])
                indexi+=1
            sorti = sorted(dist_values, key=SortValue)
            sorti = sorti[:k]
            K_classes = getclass(sorti,self.traininglabels)
            ranked = highestclass(K_classes)
            cl =[]
            for i in range(len(ranked)):
                cl.append(ranked[i][0])
            co = pd.DataFrame(columns=cl)
            h= data.columns.difference(co.columns)
            for d in h:
                data.loc[incre:incre,d] = 0
            b = data.columns.intersection(co.columns)
            for v in b:
                for i in range(len(ranked)):
                    if (v == ranked[i][0]):
                        data.loc[incre:incre,v] = (ranked[i][1]/k)
            incre +=1
            pclass = finalclass(ranked)
            predicted.append(pclass)
        return data


In [4]:


glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)]
results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

results

Training time: 0.03 s.
Testing time (k=1): 1.03 s.
Testing time (k=3): 0.84 s.
Testing time (k=5): 0.69 s.
Testing time (k=7): 0.70 s.
Testing time (k=9): 0.89 s.


,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.474019,0.833805
7,0.598131,0.470723,0.834465
9,0.616822,0.483674,0.828734
